In [1]:
import requests
import pandas as pd
import time
import json
import ast
import numpy as np
import glob

In [2]:
from ta import trend
from ta import momentum

In [3]:
def convert_to_json(text):
    try:
        return ast.literal_eval(text)
    except:
#         print(text)
        return {}

In [4]:
def read_all_file_in_directory(directory, file_type, sep = "\t"):
    print(directory)
    file_list= [f for f in glob.glob(directory + "*." + file_type)]
    
    all_data = pd.DataFrame()
    temp = pd.DataFrame()
    for file in file_list:
            df = pd.read_csv(file, sep= sep, encoding = "utf8", engine='python')
            temp = pd.concat([temp, df])
    all_data = pd.concat([all_data, temp])
    return all_data

In [13]:
# def clean_candlestick_data(df, price_type = 'ask' ):
#     df_modified = df[[ 'currency_pair', 'time', 'volume', price_type]]
    
#     df_modified[price_type] = df_modified[price_type].apply(convert_to_json)
#     price_df = pd.json_normalize(df_modified[price_type])
    
#     df_modified = pd.merge(df_modified, price_df,
#                           left_index=True, right_index=True)
    
#     df_modified = df_modified[['currency_pair', 'time', 'volume','o', 'h', 'l', 'c']]
#     df_modified.columns = ['currency_pair', 'time', 'volume','price_open','price_high', 'price_low', 'price_close']
    
#     df_modified["price_open"] = df_modified["price_open"].astype(float)
#     df_modified["price_high"] = df_modified["price_high"].astype(float)
#     df_modified["price_low"] = df_modified["price_low"].astype(float)
#     df_modified["price_close"] = df_modified["price_close"].astype(float)
#     df_modified["time"] = pd.to_datetime(df_modified["time"])
    
#     return(df_modified)


def clean_candle_data(df, price_type = 'ask' ):
    
    df_modified = df[['complete', 'currency_pair', 'time', 'volume', price_type]]
    df_modified[price_type] = df_modified[price_type].astype(str)
    df_modified[price_type] = df_modified[price_type].apply(convert_to_json)
    
    price_df = pd.json_normalize(df_modified[price_type])
    
    df_modified = pd.merge(df_modified, price_df,
                          left_index=True, right_index=True)
    print(df_modified.columns)
    
    df_modified = df_modified[['complete','currency_pair', 'time', 'volume','o', 'h', 'l', 'c']]
    df_modified.columns = ['complete','currency_pair', 'time', 'volume','price_open','price_high', 'price_low', 'price_close']
    
    df_modified[["price_open", "price_high", "price_low", "price_close"]] = df_modified[["price_open", "price_high",
                                                                                         "price_low", "price_close"]].apply(pd.to_numeric)
    df_modified["time"] = pd.to_datetime(df_modified["time"])
    
    return df_modified

In [6]:
def get_macd_indicator(df, window_slow: int = 26, window_fast: int = 12, window_sign: int = 9, fillna: bool = False):
    # Innitialising MACD indicator
    indicator_macd = trend.MACD(close = df["price_close"])
    # Create columns for MACCD line, singal line and macd histogram (macd crosses signal line)
    df['macd'] = indicator_macd.macd()
    df['macd_signal'] = indicator_macd.macd_signal()
    df['macd_signal_diff'] = indicator_macd.macd_diff()
    return df

In [7]:
"""
Create an empty dictionary to store time of potential trades (buy and sell)
If the dictionary is empty AND
if the ccurrent macd_signal_diff value is NOT nan then
store the fist non-nan value
"""
"""
If the dictionary is NOT empty AND
if the current macd_signal_diff is > 0 AND
if the previous macd_signal_diff is < 0 THEN
store the current macd_signal_diff value
"""
"""
If the dictionary is NOT empty AND
if the current macd_signal_diff is < 0 AND
if the previous macd_signal_diff is > 0 THEN
store the current macd_signal_diff value
"""

def macd_buy_sell_signal(df):
    
    df = get_macd_indicator(df)
    
    macd_signal_dict = {}
    for i in range(0,len(df)):
        if bool(macd_signal_dict) == False:
            if np.isnan(df["macd_signal_diff"][i]) == False: 
                macd_signal_dict[df["time"][i]] = df["macd_signal_diff"][i]    
        else:
            if df["macd_signal_diff"][i] > 0:
                if list(macd_signal_dict.values())[-1] < 0:
                    macd_signal_dict[df["time"][i]] = df["macd_signal_diff"][i]

            elif df["macd_signal_diff"][i] < 0:
                if list(macd_signal_dict.values())[-1] > 0:
                    macd_signal_dict[df["time"][i]] = df["macd_signal_diff"][i]

    macd_signal_df= pd.DataFrame(macd_signal_dict.items())
    macd_signal_df.columns = ["time", "macd_signal_value"]
    macd_signal_df['buy_sell_signal'] = np.where(macd_signal_df['macd_signal_value'] > 0, "buy", "sell")
    
    df = pd.merge(df, macd_signal_df,
                      on ="time",
                      how = "left")
    return df              

In [8]:
def trade_execution(df, order_type, macd_signal_time, take_profit_pip = 10, stop_loss_pip = 10):
    
    df["index"] = df.index
    signal_time_index = df.loc[df['time'] == macd_signal_time, 'index'].iloc[0]
    order_time_index = signal_time_index + 1
    order_price_start = df["price_open"][order_time_index]
    order_time_start = df["time"][order_time_index]

    if order_type == "buy":
        price_tp = round(order_price_start + (take_profit_pip/10000),5)
        price_sl = round(order_price_start - (stop_loss_pip/10000),5)
#         print(order_type, order_time_start, "start:" + str(order_price_start), "tp:" + str(price_tp), "sl:" + str(price_sl) )
        for index in range(order_time_index, len(df)):
            current_price_low = round(df["price_low"][index],5)
            current_price_high = round(df["price_high"][index],5)
            current_time = df["time"][index]
#             print(order_type, current_time, "start:" + str(order_price_start), "low:" + str(current_price_low), "high:" + str(current_price_high) )
            if (current_price_low <= price_sl) and (current_price_high >= price_tp):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Not Determined"}, index=[0])
                return result_df
                break
            elif (current_price_low <= price_sl):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Stop Loss Order Executed"}, index=[0])
                return result_df
                break
            elif (current_price_high >= price_tp):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Take Profit Order Executed"}, index=[0])
                return result_df
                break
            else:
                continue

    elif order_type == "sell":
        price_tp = round(order_price_start - (take_profit_pip/10000),5)
        price_sl = round(order_price_start + (stop_loss_pip/10000),5)
#         print(order_type, order_time_start, "start:" + str(order_price_start), "tp:" + str(price_tp), "sl:" + str(price_sl) )

        for index in range(order_time_index, len(df)):
            current_price_low = round(df["price_low"][index],5)
            current_price_high = round(df["price_high"][index],5)
            current_time = df["time"][index]
#             print(order_type, current_time, "start:" + str(order_price_start), "low:" + str(current_price_low), "high:" + str(current_price_high) )

            if (current_price_low <= price_tp) and (current_price_high >= price_sl):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Not Determined"}, index=[0])
                return result_df
                break
            elif (current_price_low <= price_tp):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Take Profit Order Executed"}, index=[0])
                return result_df
                break
            elif (current_price_high >= price_sl):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Stop Loss Order Executed"}, index=[0])
                return result_df
                break
            else:
                continue
    else:
        return("Invalid Order Type (only buy or sell)")

    

#### READ AND CLEAN PRICE DATA

In [9]:
directory = "/Users/nguyenhoangnam/Documents/Python/Stock - Forex Investing & Trading/Forex/OANDA/"
curency_pair = "EURUSD"
time_range = "M15"
path = directory + curency_pair + "/" +  time_range + "/Raw Data/"


df = read_all_file_in_directory(path, file_type = "csv")
df.shape

/Users/nguyenhoangnam/Documents/Python/Stock - Forex Investing & Trading/Forex/OANDA/EURUSD/M15/Raw Data/


(285000, 6)

In [10]:
df_clean = clean_candlestick_data(df)

<ipython-input-5-30396012ae1a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modified[price_type] = df_modified[price_type].apply(convert_to_json)


In [12]:
df

,complete,volume,time,bid,ask,currency_pair
0,True,3,2010-01-03T17:45:00.000000000Z,"{'o': '1.43070', 'h': '1.43097', 'l': '1.43045...","{'o': '1.43170', 'h': '1.43247', 'l': '1.43170...",EUR_USD
1,True,36,2010-01-03T18:00:00.000000000Z,"{'o': '1.43097', 'h': '1.43325', 'l': '1.43005...","{'o': '1.43247', 'h': '1.43525', 'l': '1.43185...",EUR_USD
2,True,6,2010-01-03T18:15:00.000000000Z,"{'o': '1.43075', 'h': '1.43080', 'l': '1.43055...","{'o': '1.43225', 'h': '1.43230', 'l': '1.43205...",EUR_USD
3,True,40,2010-01-03T18:30:00.000000000Z,"{'o': '1.43100', 'h': '1.43165', 'l': '1.43031...","{'o': '1.43250', 'h': '1.43315', 'l': '1.43181...",EUR_USD
4,True,55,2010-01-03T18:45:00.000000000Z,"{'o': '1.43036', 'h': '1.43097', 'l': '1.43031...","{'o': '1.43186', 'h': '1.43222', 'l': '1.43177...",EUR_USD
...,...,...,...,...,...,...
284995,True,1573,2021-05-04T15:45:00.000000000Z,"{'o': '1.20207', 'h': '1.20215', 'l': '1.20115...","{'o': '1.20219', 'h': '1.20228', 'l': '1.20126...",EUR_USD
284996,True,1672,2021-05-04T16:00:00.000000000Z,"{'o': '1.20139', 'h': '1.20239', 'l': '1.20120...","{'o': '1.20151', 'h': '1.20251', 'l': '1.20130...",EUR_USD
284997,True,1207,2021-05-04T16:15:00.000000000Z,"{'o': '1.20207', 'h': '1.20207', 'l': '1.20156...","{'o': '1.20218', 'h': '1.20218', 'l': '1.20169...",EUR_USD
284998,True,1204,2021-05-04T16:30:00.000000000Z,"{'o': '1.20164', 'h': '1.20182', 'l': '1.20106...","{'o': '1.20177', 'h': '1.20194', 'l': '1.20119...",EUR_USD


#### STRATERGY 1: MACD CROSSOVER

In [25]:
# Get MACD buy and sell signals
df_with_macd_signals = macd_buy_sell_signal(df_clean)

In [26]:
indicator_sma = trend.SMAIndicator(close = df_with_macd_signals["price_close"], window=200)
df_with_macd_signals['sma_200'] = indicator_sma.sma_indicator()

In [27]:
indicator_rsi = momentum.RSIIndicator(close = df_with_macd_signals["price_close"], window=14)
df_with_macd_signals['rsi_14'] = indicator_rsi.rsi()

In [28]:
def filter_trade_signals(df, rsi_oversold, rsi_overbought):
    df["new_buy_sell_signal"] = ""
    for i in range(len(df)-3):
        if (df["buy_sell_signal"][i] == "buy") and (df["macd_signal_diff"][i+1] > 0)  and (df["price_close"][i+1] > df["sma_200"][i+1]) and (df["rsi_14"][i+1] < rsi_overbought):
            df["new_buy_sell_signal"][i+1] = "buy"
        elif (df["buy_sell_signal"][i] == "sell") and (df["macd_signal_diff"][i+1] < 0) and (df["price_close"][i+1] < df_with_macd_signals["sma_200"][i+1]) and (df["rsi_14"][i+1] > rsi_oversold):
            df["new_buy_sell_signal"][i+1] = "sell"
#     print(df["buy_sell_signal"].value_counts())
#     print(df["new_buy_sell_signal"].value_counts())
    return df
            

In [29]:
def get_trade_results(df):

    df_with_macd_signals_filtered = df[df["new_buy_sell_signal"].isin(["buy", "sell"])].reset_index(drop=True)
    df_with_macd_signals_filtered.shape

    

    output_df = pd.DataFrame()
    for i in range(0,len(df_with_macd_signals_filtered)-1):
        output = trade_execution(df, df_with_macd_signals_filtered["new_buy_sell_signal"][i],
                                df_with_macd_signals_filtered["time"][i], 10,10)
        output_df = output_df.append(output)
    return output_df
    



     

In [30]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
results_df = []
for oversold_value in range(25,100,5):
    for overbought_value in range(75,0,-5):
        
        df_with_macd_signals = filter_trade_signals(df_with_macd_signals, oversold_value,overbought_value)
        output_df = get_trade_results(df_with_macd_signals)
        total_trade = len(output_df)
        try:
          winner_count = len(output_df[output_df["order_outcome"] == "Take Profit Order Executed"])
        except:
          winner_count = 0
        try:
          loser_count = len(output_df[output_df["order_outcome"] == "Stop Loss Order Executed"])
        except:
          loser_count = 0
        try:
          win_rate = winner_count/total_trade*100
        except:
          win_rate = 0
        
        print(oversold_value, overbought_value,win_rate,total_trade)
        temp = pd.DataFrame(
            {'oversold_value': oversold_value, 'overbought_value': overbought_value,
             'win_rate': win_rate, 'total_trade': total_trade, 'winner_count': winner_count, 'loser_count': loser_count}, index=[0])
        results_df.append(temp)
results_df = pd.concat(results_df, ignore_index=True) 

25 75 48.96320390323237 9838
25 70 49.21135646687697 9510
25 65 49.40167080605103 8858
25 60 49.81036662452591 7910
25 55 49.76140769460185 6706
25 50 49.47716974555594 5738
25 45 48.945637454053006 5169
25 40 49.018814485130484 4943
25 35 48.9161554192229 4890
25 30 48.89480147359804 4886
25 25 48.89480147359804 4886
25 20 48.89480147359804 4886
25 15 48.89480147359804 4886
25 10 48.89480147359804 4886
25 5 48.89480147359804 4886
30 75 48.87162800461845 9527
30 70 49.12490488096532 9199
30 65 49.31554931554932 8547
30 60 49.73022766153441 7599
30 55 49.66379984362783 6395
30 50 49.34586327621153 5427
30 45 48.76492383696995 4858
30 40 48.83419689119171 4632
30 35 48.72242847783359 4579
30 30 48.69945355191257 4575
30 25 48.69945355191257 4575
30 20 48.69945355191257 4575
30 15 48.69945355191257 4575
30 10 48.69945355191257 4575
30 5 48.69945355191257 4575
35 75 49.056603773584904 8851
35 70 49.337087879854515 8523
35 65 49.561682124253586 7871
35 60 50.05055611729019 6923
35 55 50.043

In [31]:
results_df["profit"] = (results_df["win_rate"] - 50) * results_df["total_trade"]

In [32]:
results_df = results_df.sort_values("profit", ascending= False)

In [35]:
results_df['currency_pair'] = curency_pair
results_df['time_range'] = time_range

In [36]:
results_df

,oversold_value,overbought_value,win_rate,total_trade,winner_count,loser_count,profit,currency_pair,time_range
108,60,60,51.383914,3071,1578,1464,4250.0,EURUSD,M15
109,60,55,52.222817,1867,975,875,4150.0,EURUSD,M15
123,65,60,51.271886,3027,1552,1446,3850.0,EURUSD,M15
168,80,60,51.273569,3023,1550,1444,3850.0,EURUSD,M15
183,85,60,51.273569,3023,1550,1444,3850.0,EURUSD,M15
...,...,...,...,...,...,...,...,...,...
1,25,70,49.211356,9510,4680,4694,-7500.0,EURUSD,M15
16,30,70,49.124905,9199,4519,4556,-8050.0,EURUSD,M15
30,35,75,49.056604,8851,4342,4390,-8350.0,EURUSD,M15
0,25,75,48.963204,9838,4817,4874,-10200.0,EURUSD,M15


In [38]:
df.to_csv("/Users/nguyenhoangnam/Downloads/2010-01-01_2021-05-01_eur_usd_m15_strategy_1_win_rates.csv", encoding="utf8", index=None, sep="\t")

In [41]:
results_df.head(50)

,oversold_value,overbought_value,win_rate,total_trade,winner_count,loser_count,profit,currency_pair,time_range
108,60,60,51.383914,3071,1578,1464,4250.0,EURUSD,M15
109,60,55,52.222817,1867,975,875,4150.0,EURUSD,M15
123,65,60,51.271886,3027,1552,1446,3850.0,EURUSD,M15
168,80,60,51.273569,3023,1550,1444,3850.0,EURUSD,M15
183,85,60,51.273569,3023,1550,1444,3850.0,EURUSD,M15
138,70,60,51.273569,3023,1550,1444,3850.0,EURUSD,M15
153,75,60,51.273569,3023,1550,1444,3850.0,EURUSD,M15
198,90,60,51.273569,3023,1550,1444,3850.0,EURUSD,M15
213,95,60,51.273569,3023,1550,1444,3850.0,EURUSD,M15
93,55,60,51.171281,3287,1682,1571,3850.0,EURUSD,M15


In [ ]:
reviews_df["scenario_number"] = reviews_df.index +1

In [ ]:
test = reviews_df[reviews_df["win_rate"] == reviews_df["win_rate"].max()]

In [ ]:
test

In [ ]:
scenario_number_list = reviews_df["scenario_number"].to_list()
win_rate_list = reviews_df["scenario_number"].to_list()

In [ ]:
scenario_number

In [ ]:
import matplotlib.pyplot as plt

plt.plot(reviews_df["scenario_number"],reviews_df["win_rate"])
plt.title('title name')
plt.xlabel('xAxis name')
plt.ylabel('yAxis name')
plt.show()

In [ ]:
df_with_macd_signals["buy_sell_signal"].value_counts()

In [ ]:
df_with_macd_signals["new_buy_sell_signal"].value_counts()

In [ ]:
df_with_macd_signals_filtered = df_with_macd_signals[df_with_macd_signals["new_buy_sell_signal"].isin(["buy", "sell"])].reset_index(drop=True)
df_with_macd_signals_filtered.shape

In [ ]:
# trade at [i]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+2]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+3]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+4]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+5]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i] and above/below SMA200
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] and above/below SMA200
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+2] and above/below SMA200
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] AND above/below SMA200 AND RSI (30, 70)
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] AND above/below SMA200 AND RSI (35, 65)
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] AND above/below SMA200 AND RSI (40, 60)
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] AND above/below SMA200 AND RSI (40, 60)
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
df_with_macd_signals["new_buy_sell_signal"] = ""
for i in range(len(df_with_macd_signals)-3):
    if (df_with_macd_signals["buy_sell_signal"][i] == "buy") and (df_with_macd_signals["macd_signal_diff"][i+1] > 0)  and (df_with_macd_signals["price_close"][i+1] > df_with_macd_signals["sma_200"][i+1]) and (df_with_macd_signals["rsi_14"][i+1] < 60):
        df_with_macd_signals["new_buy_sell_signal"][i+1] = "buy"
    elif (df_with_macd_signals["buy_sell_signal"][i] == "sell") and (df_with_macd_signals["macd_signal_diff"][i+1] < 0) and (df_with_macd_signals["price_close"][i+1] < df_with_macd_signals["sma_200"][i+1]) and (df_with_macd_signals["rsi_14"][i+1] > 40):
        df_with_macd_signals["new_buy_sell_signal"][i+1] = "sell"
        

In [ ]:
df["order_outcome"].value_counts()

In [ ]:
pd.DataFrame({"count": df.groupby(["order_type", "order_outcome"]).size()}).reset_index().sort_values("order_type", ascending = False)
